#### Build zoningmods lookup table for PBA50+ and add Draft Blueprint zoning values

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import pathlib
import os

In [2]:
HOME_DIR = pathlib.Path.home()

In [3]:
M_DRIVE = pathlib.Path("/Volumes/Data/Models") if os.name != "nt" else pathlib.Path("M:/")
BOX_DRIVE = HOME_DIR / 'Library/CloudStorage/Box-Box'

Build the PBA50+ zoningmods table using the parcels geography table and "zoningmodcat" column

In [4]:
parcels_geography_fbp_path = M_DRIVE / "urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/fbp_urbansim_growth_geographies.csv"
parcels_classes_fbp_path = M_DRIVE / "urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/fbp_urbansim_parcel_classes.csv"

In [5]:
# urbansim_parcels_path = BOX_DRIVE / 'Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/BASIS/PBA50Plus/urbansim_geodata.gpkg' 
# urbansim_parcels_path_csv = BOX_DRIVE / 'Modeling and Surveys/Urban Modeling/Bay Area UrbanSim/BASIS/PBA50Plus/urbansim_parcels_topo_fix.csv' 
# # urbansim_parcels_path = M_DRIVE / "urban_modeling/baus/BAUS Inputs/basis_inputs/crosswalks/urbansim_geodata.gpkg"
# gpd.list_layers(urbansim_parcels_path)

# %%time
# urbansim_parcels = gpd.read_file(urbansim_parcels_path,engine='pyogrio', layer='urbansim_parcels_topo_fix')

In [6]:
# read in FBP parcels geography
parcels_geog = pd.read_csv(parcels_classes_fbp_path)

In [7]:
# list the columns defining the parcel classes

zoning_mod_cols = ["gg_id", "exd_id", "tra_id", "hra_id", "ppa_id", "ugb_id"]

In [8]:
# check the core classifying columns
parcels_geog[zoning_mod_cols] = parcels_geog[zoning_mod_cols].astype(str)
parcels_geog[zoning_mod_cols]

,gg_id,exd_id,tra_id,hra_id,ppa_id,ugb_id
0,nan,exd,nan,nan,nan,nan
1,nan,nan,nan,nan,nan,nan
2,GG,nan,tra_5,nan,nan,UGB
3,GG,nan,nan,nan,PPA,UGB
4,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...
1956202,nan,exd,nan,nan,nan,UGB
1956203,nan,exd,nan,HRA,nan,UGB
1956204,nan,exd,nan,HRA,nan,UGB
1956205,nan,exd,nan,nan,nan,UGB


In [9]:
# assign concatenations of the component columns

parcels_geog["zoningmodcat"] = (
    parcels_geog[zoning_mod_cols]
    .astype(str)
    .apply(lambda x: "".join(x), axis=1)
    .str.lower()
)

In [11]:

parcels_geog['group_id']=parcels_geog.groupby(["zoningmodcat"]+zoning_mod_cols).ngroup()
parcels_geog.head()

,parcel_id,gg_id,pda_id,ppa_id,tra_id,hra_id,ugb_id,tpp_id,dis_id,exp2020_id,exsfd_id,exd_id,zoningmodcat,group_id
0,1972208.0,nan,NaN,nan,nan,nan,nan,NaN,NaN,exp1,NaN,exd,nanexdnannannannan,65
1,2003758.0,nan,NaN,nan,nan,nan,nan,NaN,DIS,out,NaN,nan,nannannannannannan,78
2,1713936.0,GG,Gilroy - Downtown Gilroy,nan,tra_5,nan,UGB,b2,NaN,in,NaN,nan,ggnantra_5nannanugb,54
3,287094.0,GG,NaN,PPA,nan,nan,UGB,b2,NaN,in,NaN,nan,ggnannannanppaugb,35
4,2004034.0,nan,NaN,nan,nan,nan,nan,NaN,DIS,out,NaN,nan,nannannannannannan,78


In [13]:
parcels_geog.groupby(["zoningmodcat"]).size().sort_values()

zoningmodcat
ggexdnanhranannan           1
ggexdnanhrappaugb           1
ggexdtra_3nanppaugb         1
ggexdtra_4hrappaugb         1
ggnantra_5nanppanan         1
                        ...  
ggexdtra_5hrananugb     69208
ggexdtra_4hrananugb     96991
ggexdtra_4nannanugb    131529
nanexdnanhrananugb     439471
nanexdnannannanugb     605226
Length: 90, dtype: int64

# Build the zoning mods

Add Zoning Step 1: Add unincorporated UGB zoning (doesn't apply to PPAs)- Applied to No Project but not Draft Blueprint

In [14]:
# if within UGB but in unincorporated area, some upzoning is allowed for historic expansion
# Non-Residential building on parcel
#zoningmods.loc[(zoningmods.ugb_id == 'Uninc UGB') & (zoningmods.exd_id != 'exd'), 'dua_up'] = 2.75
#zoningmods.loc[(zoningmods.ugb_id == 'Uninc UGB') & (zoningmods.exd_id != 'exd'), 'add_bldg'] = 'HS'
# Residential building on parcel
#zoningmods.loc[(zoningmods.ugb_id == 'Uninc UGB') & (zoningmods.exd_id == 'exd'), 'dua_up'] = 1.5
#zoningmods.loc[(zoningmods.ugb_id == 'Uninc UGB') & (zoningmods.exd_id == 'exd'), 'add_bldg'] = 'HS'

Step 2: Add residential and non-residential growth geography upzoning, which can override unincorporated UGB zoning

In [16]:
# First we define the universe: Growth Geographies (GG) with different area classifications
# - TRA: Transit-Rich Area (TRA1, TRA2, TRA3, TRA4, TRA5, TRA6)
# - HRA: High Resource Areas 
# - EXD: Residential Parcel (exd_id == 'exd' means it is a residential parcel)
# create a df with unique combinations of the components - and zoningmodcat

zoningmods = parcels_geog.groupby(['zoningmodcat']+zoning_mod_cols).size().reset_index(name='count')

# Check column domains for reference
for col in zoning_mod_cols:
    print(col)
    print(zoningmods[col].unique())
    print('-'*80)

# Define common expressions for filtering
is_gg = "gg_id == 'GG'"
is_tra1 = "tra_id == 'tra_1'"
is_tra2 = "tra_id == 'tra_2'"
is_tra3 = "tra_id == 'tra_3'"
is_tra4 = "tra_id == 'tra_4'"
is_tra5 = "tra_id == 'tra_5'"
is_tra6 = "tra_id == 'tra_6'"
is_hra = "hra_id == 'HRA'"
is_not_hra = "hra_id != 'HRA'"
is_exd = "exd_id == 'exd'"
is_not_exd = "exd_id != 'exd'"
is_no_tra = "tra_id=='nan'"
is_no_hra = "hra_id=='nan'"
is_ppa = "ppa_id=='PPA'"
is_ugb = "ugb_id=='UGB'"


# List of building types to add for least restrictive areas
building_list_inclusive = 'HT HM OF HO SC RS RB MR MT ME'

# list of residential types to add for residential areas
building_list_res = 'HM MR'

# list of columns to update for nonres, res sets
non_res_cols = ['dua_up', 'far_up', 'add_bldg']
res_cols = ['dua_up', 'add_bldg']

# Apply updates based on conditions

# initialze the three colums to set
zoningmods[non_res_cols] = 0,0.0,''
zoningmods['drop_bldg'] =''


# Apply updates using masks

# GG + TRA1 + HRA
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra1} & {is_hra} & {is_not_exd}"), ['dua_up', 'far_up']] = 300, 30  
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra1} & {is_hra} & {is_exd}"), ['dua_up']] = 50 

# GG + TRA1 (no HRA)
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra1} & {is_not_hra} & {is_not_exd}"), ['dua_up', 'far_up']] = 250, 30  
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra1} & {is_not_hra} & {is_exd}"), ['dua_up']] = 50 

# GG + TRA2 + HRA
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra2} & {is_hra} & {is_not_exd}"), ['dua_up', 'far_up']] = 250, 25  
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra2} & {is_hra} & {is_exd}"), ['dua_up']] = 50 

# GG + TRA2 (no HRA)
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra2} & {is_not_hra} & {is_not_exd}"), ['dua_up', 'far_up']] = 200, 25  
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra2} & {is_not_hra} & {is_exd}"), ['dua_up']] = 35 

# GG + TRA3 + HRA
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra3} & {is_hra} & {is_not_exd}"), ['dua_up', 'far_up']] = 200, 15  
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra3} & {is_hra} & {is_exd}"), ['dua_up']] = 50 

# GG + TRA3 (no HRA)
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra3} & {is_not_hra} & {is_not_exd}"), ['dua_up', 'far_up']] = 150, 15  
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra3} & {is_not_hra} & {is_exd}"), ['dua_up']] = 30 

# GG + TRA4 + HRA
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra4} & {is_hra} & {is_not_exd}"), ['dua_up', 'far_up']] = 100, 5  
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra4} & {is_hra} & {is_exd}"), ['dua_up']] = 50 

# GG + TRA4 (no HRA)
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra4} & {is_not_hra} & {is_not_exd}"), ['dua_up', 'far_up']] = 100, 5  
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra4} & {is_not_hra} & {is_exd}"), ['dua_up']] = 30 

# GG + TRA5 + HRA
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra5} & {is_hra} & {is_not_exd}"), ['dua_up', 'far_up']] = 75, 2.5  
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra5} & {is_hra} & {is_exd}"), ['dua_up']] = 30 

# GG + TRA5 (no HRA)
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra5} & {is_not_hra} & {is_not_exd}"), ['dua_up', 'far_up']] = 50, 2.5  
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra5} & {is_not_hra} & {is_exd}"), ['dua_up']] = 25  

# GG + TRA6 
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra6} & {is_not_exd}"), ['dua_up', 'far_up']] = 35, 0  
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_tra6} & {is_exd}"), ['dua_up']] = 0 

# GG + PPA  (NEW - Corrected values and conditions)
zoningmods.loc[zoningmods.eval(f"{is_gg} & {is_ppa} & {is_not_exd}"), ['far_up']] =  0.35 



zoningmods

gg_id
['GG' 'nan']
--------------------------------------------------------------------------------
exd_id
['exd' 'nan']
--------------------------------------------------------------------------------
tra_id
['nan' 'tra_1' 'tra_2' 'tra_3' 'tra_4' 'tra_5' 'tra_6']
--------------------------------------------------------------------------------
hra_id
['HRA' 'nan']
--------------------------------------------------------------------------------
ppa_id
['nan' 'PPA']
--------------------------------------------------------------------------------
ugb_id
['nan' 'UGB']
--------------------------------------------------------------------------------


,zoningmodcat,gg_id,exd_id,tra_id,hra_id,ppa_id,ugb_id,count,dua_up,far_up,add_bldg,drop_bldg
0,ggexdnanhranannan,GG,exd,nan,HRA,nan,nan,1,0,0.0,,
1,ggexdnanhrananugb,GG,exd,nan,HRA,nan,UGB,5799,0,0.0,,
2,ggexdnanhrappaugb,GG,exd,nan,HRA,PPA,UGB,1,0,0.0,,
3,ggexdnannannannan,GG,exd,nan,nan,nan,nan,49,0,0.0,,
4,ggexdnannannanugb,GG,exd,nan,nan,nan,UGB,25451,0,0.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...
85,nannantra_5nannanugb,nan,nan,tra_5,nan,nan,UGB,99,0,0.0,,
86,nannantra_6hranannan,nan,nan,tra_6,HRA,nan,nan,2,0,0.0,,
87,nannantra_6hrananugb,nan,nan,tra_6,HRA,nan,UGB,331,0,0.0,,
88,nannantra_6nannannan,nan,nan,tra_6,nan,nan,nan,79,0,0.0,,


Step 3: Add PPA zoning, which can override unincorporated UGB zoning and growth geography upzoning

In [19]:
# if within PPA, add industrial zoning and "drop_bldg" for all other building types

zoningmods.loc[zoningmods.eval(is_ppa), 'far_up'] = 2
zoningmods.loc[zoningmods.eval(is_ppa), 'add_bldg'] = 'IW'
zoningmods.loc[zoningmods.eval(is_ppa), 'drop_bldg'] = 'HT HM OF HO SC RS RB MR MT ME'

# also set "dua_up" and "add_bldg" to nan
zoningmods.loc[zoningmods.eval(is_ppa), 'dua_up'] = np.nan
zoningmods.loc[zoningmods.eval(is_ppa), 'add_bldg'] = np.nan
zoningmods.loc[zoningmods.eval(is_ppa)]

,zoningmodcat,gg_id,exd_id,tra_id,hra_id,ppa_id,ugb_id,count,dua_up,far_up,add_bldg,drop_bldg
2,ggexdnanhrappaugb,GG,exd,nan,HRA,PPA,UGB,1,NaN,2.0,NaN,HT HM OF HO SC RS RB MR MT ME
5,ggexdnannanppaugb,GG,exd,nan,nan,PPA,UGB,898,NaN,2.0,NaN,HT HM OF HO SC RS RB MR MT ME
12,ggexdtra_3nanppaugb,GG,exd,tra_3,nan,PPA,UGB,1,NaN,2.0,NaN,HT HM OF HO SC RS RB MR MT ME
14,ggexdtra_4hrappaugb,GG,exd,tra_4,HRA,PPA,UGB,1,NaN,2.0,NaN,HT HM OF HO SC RS RB MR MT ME
17,ggexdtra_4nanppaugb,GG,exd,tra_4,nan,PPA,UGB,24,NaN,2.0,NaN,HT HM OF HO SC RS RB MR MT ME
20,ggexdtra_5hrappaugb,GG,exd,tra_5,HRA,PPA,UGB,15,NaN,2.0,NaN,HT HM OF HO SC RS RB MR MT ME
23,ggexdtra_5nanppaugb,GG,exd,tra_5,nan,PPA,UGB,36,NaN,2.0,NaN,HT HM OF HO SC RS RB MR MT ME
28,ggexdtra_6nanppaugb,GG,exd,tra_6,nan,PPA,UGB,7,NaN,2.0,NaN,HT HM OF HO SC RS RB MR MT ME
31,ggnannanhrappaugb,GG,nan,nan,HRA,PPA,UGB,154,NaN,2.0,NaN,HT HM OF HO SC RS RB MR MT ME
34,ggnannannanppanan,GG,nan,nan,nan,PPA,nan,20,NaN,2.0,NaN,HT HM OF HO SC RS RB MR MT ME


Step 4: Add UGB zoning, which can override unincorporated UGB, growth geography, and PPA upzoning

In [20]:
# if outside of UGB "far_down" and "dua_down" are 0

zoningmods.loc[zoningmods.eval(is_ugb), ['dua_down', 'far_down']] = 0

# and also set other "dua_up", "far_up", and "add_bldg" to nan
zoningmods.loc[zoningmods.eval(is_ugb), ['far_up','dua_up', 'add_bldg', 'drop_bldg']] = np.nan
zoningmods.loc[zoningmods.eval(is_ugb)]

,zoningmodcat,gg_id,exd_id,tra_id,hra_id,ppa_id,ugb_id,count,dua_up,far_up,add_bldg,drop_bldg,dua_down,far_down
1,ggexdnanhrananugb,GG,exd,nan,HRA,nan,UGB,5799,NaN,NaN,NaN,NaN,0.0,0.0
2,ggexdnanhrappaugb,GG,exd,nan,HRA,PPA,UGB,1,NaN,NaN,NaN,NaN,0.0,0.0
4,ggexdnannannanugb,GG,exd,nan,nan,nan,UGB,25451,NaN,NaN,NaN,NaN,0.0,0.0
5,ggexdnannanppaugb,GG,exd,nan,nan,PPA,UGB,898,NaN,NaN,NaN,NaN,0.0,0.0
6,ggexdtra_1hrananugb,GG,exd,tra_1,HRA,nan,UGB,2465,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,nannantra_4hrananugb,nan,nan,tra_4,HRA,nan,UGB,22,NaN,NaN,NaN,NaN,0.0,0.0
83,nannantra_4nannanugb,nan,nan,tra_4,nan,nan,UGB,56,NaN,NaN,NaN,NaN,0.0,0.0
85,nannantra_5nannanugb,nan,nan,tra_5,nan,nan,UGB,99,NaN,NaN,NaN,NaN,0.0,0.0
87,nannantra_6hrananugb,nan,nan,tra_6,HRA,nan,UGB,331,NaN,NaN,NaN,NaN,0.0,0.0


In [22]:
# finally, replace string 'nan' with the real thing

zoningmods.loc[:,zoning_mod_cols] = zoningmods.loc[:,zoning_mod_cols].replace('nan',np.nan)

In [23]:
# export
output_path = M_DRIVE / "urban_modeling/baus/BAUS Inputs/plan_strategies/zoning_mods_PBA50Plus_FBP.csv"
zoningmods.to_csv(output_path, index=False)